# Implementing some abstractions
At this stage, I claim we've stumbled on a pattern that we might want to re-use. Vega / lite publish a [comprehensive](https://vega.github.io/vega/examples/) set of [examples](https://vega.github.io/vega-lite/examples/) - we want to start from them, modify them, and plot our chart in some "context". 

One idea would be to [write a script](https://github.com/Quafadas/dedav4s/blob/main/core/jvm/src/main/scala/viz/genVegaPlots.sc) which scrapes the ~~examples~~ [links to the examples](https://github.com/Quafadas/dedav4s/blob/main/core/shared/src/main/scala/viz/vega/plots/SpecUrls.scala) off the vega/lite website. Given that, you need only a very simple http library, to go fetch those battle-tested-part-of-the-core-vega-library starting points.

We'll pragmatically accept mutability and [iterate around](https://github.com/Quafadas/dedav4s/blob/861c3fa38f41084f9d2e1ea168da40aab22eccf5/core/shared/src/main/scala/viz/WithBaseSpec.scala#L31) a ```Seq[ujson.Value => Unit]``` (a list of side effects) as a way to apply changes to the spec. 

Context sounds a lot like `implicit` / `given` to my mind. Let's call it a ["plot target"](https://github.com/Quafadas/dedav4s/blob/861c3fa38f41084f9d2e1ea168da40aab22eccf5/core/jvm/src/main/scala/viz/PlotTarget.scala#L122)...

And with all this machinery we could write something which both abstracts away the messy business of the plot itself, and keeps the intent pretty clear. Let's plot the distrubition of html tags for some arbitrary website. 

In [3]:
import $ivy.`org.jsoup:jsoup:1.15.3`
import $ivy.`io.github.quafadas::dedav4s:0.10.1`

import io.github.quafadas.plots.SetupVega.{*, given}
import org.jsoup._
import scala.jdk.CollectionConverters._
import io.circe.syntax.*

def plotElementDist(url:String) =
  import viz.PlotTargets.almond
  val doc = Jsoup.connect(url).get
  val els = doc.body().children().select("*").asScala
  val counted = els.groupMapReduce(_.tag.toString())(_ => 1)(_ + _)
  val data = for ((label, number) <- counted)
    yield (
      category = label,
      value = number.toDouble
    )
  val pieChart = VegaPlot.fromString("""{
    "$schema": "https://vega.github.io/schema/vega-lite/v6.json",
    "title": "A Pie Chart",
    "description": "A simple pie chart with embedded data.",
    "width": 600,
    "height": 600,
    "data": {
      "values": [
        {"category": "cat1", "value": 4}
      ]
    },
    "mark": "arc",
    "encoding": {
      "theta": {"field": "value", "type": "quantitative"},
      "color": {"field": "category", "type": "nominal"}
  }}""")
  // This "PieChart" case class, is the abstraction described above.
  pieChart.plot(
    _.height := 650,
    _.width := 650,
    _.data.values := data.asJson
  )
end plotElementDist

-- Deprecation Warning: cmd3.sc:137:31 -----------------------------------------
137 |import _root_.scala.collection.JavaConverters.{
    |       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    |object JavaConverters in package scala.collection is deprecated since 2.13.0: Use `scala.jdk.CollectionConverters` instead


import $ivy.$                       

import $ivy.$                                   


import io.github.quafadas.plots.SetupVega.{*, given}

import org.jsoup._

import scala.jdk.CollectionConverters._

import io.circe.syntax.*


defined function plotElementDist

In [2]:
plotElementDist("https://google.com")
//val plot = plotElementDist("https://google.com")
// show(plot.spec) // for debugging a troublesome spec

-- Deprecation Warning: cmd2.sc:136:31 -----------------------------------------
136 |import _root_.scala.collection.JavaConverters.{
    |       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    |object JavaConverters in package scala.collection is deprecated since 2.13.0: Use `scala.jdk.CollectionConverters` instead


res2: scala.Unit | os.Path = ()

In [20]:
plotElementDist("https://www.scala-lang.org")

res19: PieChart = PieChart(
  mods = List(
    ammonite.$sess.cmd17$Helper$$Lambda$3590/0x00000008016f2e68@7ec1f41b,
    ammonite.$sess.cmd17$Helper$$Lambda$3591/0x00000008016f3268@136d214f,
    ammonite.$sess.cmd17$Helper$$Lambda$3592/0x00000008016f3668@335ba5df
  )
)

I quite like this, as data acquisition, and plotting are totally seperate concerns. The "intent" here is, to my eye, rather readable. Whilst plotting the html element counts of popular websites is... pointless, I found it extended nicely to other, more helpful problems :-).

## Elephant

There is however, a rather large elephant in the room - for this to work "for you", you have to be willing to do the legwork of learning vega. Whilst easy enough to trace the spec back, and find the "example" this refers to... I can't see a shortcut here for needing to understand and ultimately innovate on vega. That is actually a pretty fun undertaking... but it's (at least) a multiple day time investment. (A worthwhile one, I claim!)

In the end, this isn't realllllly ... "plotting in scala"... more like using scala to write vega.